# Logisitic Regression
Adam Haile - 2/20/2025

## Data Import

In [40]:
# # Run if you don't have the packages installed
# %pip install numpy matplotlib scikit-learn pandas

In [41]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay

Let's try out the IRIS dataset again, like we did in Linear Regression. This time we want to determine the type of iris we have from our features.

In [ ]:
# Load the Iris dataset
iris = datasets.load_iris()
X = iris.data  # Features
y = iris.target  # Target labels (0, 1, 2 corresponding to the 3 iris species)

# Let's see what we have here
X, y

So we can see from our data that this time we are working with 4 features, and 3 different classes. This is fine, as SKLearn supports the ability to train a logistic regression model that can perform multi-class predicting. 

*(Under the hood, it's essentially training multiple logistic regression models that each learn a specific class. They then learn to predict "is it my class, or some other class".)*

In [43]:
# Split into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features (important for logistic regression, will help us with reaching convergence)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Train the Logistic Regression model

In [ ]:
# Train Logistic Regression model
# ovr means one-vs-rest, lbfgs is a solver that uses an iterative approach to find the best fit
clf = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=200)
clf.fit(X_train, y_train)

## Evaluate our Logistic Regression Model

In [ ]:
# Make predictions
y_pred = clf.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:\n', classification_report(y_test, y_pred))

# Confusion Matrix
disp = ConfusionMatrixDisplay.from_estimator(clf, X_test, y_test, display_labels=iris.target_names, cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

Let's interpret these results.

First thing we get is the general model accuracy. We get an accuracy of 0.9 which is pretty good!

Next we get our classification report, this is a breakdown of more nuanced ways to evaluate the model. Let's break some of these down.
1. Accuracy: $\frac{(TP + TN)}{TP + TN + FP + TN}$. Accuracy is a metric which tells us proportion of correctly classified datapoints out of all datapoints in the dataset. It works well when datasets are balanced and all categories for false classification have similar consequences (predicting a False Positive is no worse than predicting a False Negative, vise versa.)
2. Precision: $\frac{TP}{TP + FP}$. Measures how many of the positive cases are actually positive. This is good for when the consquenses for making a false positive are more costly. Ex: Spam detection, where marking an important email as spam is bad. Does not factor in the negative case at all.
3. Recall: $\frac{TP}{TP + FN}$. Measures how many positive cases were correctly identified. Better for when false negatives are more costly. Ex: Medical diagnosis where missing a disease can be deadly. 
4. F1 Score: $2 \times \frac{Precision \times Recall}{Precision + Recall}$. Balances both precision and recall. Prevents a model from becoming too conservative (High precision, low recall) or aggressive (High recall, low precision).

We can see that across the board, the model gets 90% on Accuracy, Precision, Recall, and F1 score. This is good! It means our model is generalizing well to the unseen data and is decently performant (getting a 90% or greater on models in a real-world usecase is a pretty rare thing).

## Implementing our own Logistic Regression

We've now seen how to use logistic regression, lets make it from scratch! As last time, I am going to leave some segments for you to do instead of just executing code cells down a notebook >:)

We'll use the same data as before, so no need to import any new dataset.

In [ ]:
X_train = np.c_[...((X_train.shape[0], 1)), X_train]  # TODO: Add ones to first column for our bias term
X_test = np.c_[...((X_test.shape[0], 1)), X_test]  # TODO: Add ones to first column for our bias term

<p>
<details>
<summary>Click here to reveal the answer</summary>

<pre><code>
X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_test = np.c_[np.ones((X_test.shape[0], 1)), X_test]
</code></pre>

</details>
</p>

Let's now define some helper functions. Refer back to the slides if you want some help on the structure of these functions.

In [ ]:
def sigmoid(z):
    """Compute the sigmoid function."""
    return ... # TODO: Implement the sigmoid function

def compute_loss(y, y_pred):
    """Compute binary cross-entropy loss."""
    # Some help with what this function should look like: https://koshurai.medium.com/understanding-log-loss-a-comprehensive-guide-with-code-examples-c79cf5411426
    m = 1e-9
    return ... # TODO: Implement the loss function

<p>
<details>
<summary>Click here to reveal the answer</summary>

<pre><code>
def sigmoid(z):
    """Compute the sigmoid function."""
    return 1 / (1 + np.exp(-z))

def compute_loss(y, y_pred):
    """Compute binary cross-entropy loss."""
    m = 1e-9
    return -np.mean(y * np.log(y_pred + m) + (1 - y) * np.log(1 - y_pred + m))
</code></pre>

</details>
</p>

I'll implement gradient descent for you this time, but let's break down what it's doing so you can do it the next time.

In [49]:
def gradient_descent(X, y, lr=0.1, epochs=1000):
    """Train logistic regression using gradient descent."""
    m, n = X.shape
    theta = np.zeros(n)  # Initialize weights
    
    loss_history = []
    
    for _ in range(epochs):
        z = X @ theta  # Linear combination
        y_pred = sigmoid(z)  # Apply sigmoid
        
        gradient = (1 / m) * (X.T @ (y_pred - y))  # Compute gradient
        theta -= lr * gradient  # Update weights
        
        loss = compute_loss(y, y_pred)
        loss_history.append(loss)
    
    return theta, loss_history

If you haven't taken Calc 3 yet, think of Gradient Descent as an optimization algorithm that helps find a local minimum of a function. Imagine you have a ball at the top of a valley. Instead of just rolling freely, at each step, you check the slope (gradient) of the valley at your current position and take a small step downhill. The steepness tells you how much to move and in which direction.

In machine learning, this means adjusting the model’s parameters (weights) iteratively to minimize a loss function. The gradient of the loss function tells us how to update the weights: the larger the gradient, the bigger the step. Over multiple iterations, the weights gradually move toward a minimum where the loss is minimized

Let's break this down.

```
m, n = X.shape
theta = np.zeros(n)

loss_history = []
```
First thing we are doing is initializing our weights and additional terms that we will need to help us compute gradients.

```
for _ in range(epochs):
    z = X @ theta
    y_pred = sigmoid(z)
```
This segment is the start of our iterative loop. On the first part of the loop, we are computing the matrix multiplication between our input (X) and our current weights (theta). This will give us a transformation of our input to our linear model. We are then taking the sigmoid of the linear function to get our predicted values from the input. (In this case, the predictions of our entire training set). 

```
gradient = (1 / m) * (X.T @ (y_pred - y))
```
Next, we are computing our gradient. This will tell us what direction we need to move our parameters to in order to reach our minimum and in what direction. This will take the matrix multiplcation of our original training input and the error of our prediction (y_pred - y), and multiply it by 1 / the number of features. 

```
theta -= lr * gradient
```
The theta is our newly updated model parameters. We subtract it because by default, the gradient is computing the uphill maximized option (which we don't want, we want to move down). We also multiply it by a learning rate parameter which will help us prevent undershoot/overshooting our target parameters.

```
loss = compute_loss(y, y_pred)
loss_history.append(loss)
```
Last thing we do is compute our loss. This allows us to know how well our model's predictions do compared to the actual values. We can store these for evaluation later.

Great! So now that we have all that, lets train a model!

In [50]:
theta, loss_history = gradient_descent(X_train, y_train, lr=0.1, epochs=1000)

Let's take a look at the loss history curve and see how our model did.

In [ ]:
plt.plot(loss_history)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.show()

It looks like our model hit a pretty good accuracy point at around 200 epochs and then flatlined at the same parameters/loss for the rest of the epochs. We also see that our loss history went into the **negatives**, which doesn't entirely make sense since we know that log-likelihood should always be **non-negative**. More on this later, for now let's try it out!

In [ ]:
def predict(X, theta):
    """Predict class labels (0 or 1) for input X."""
    return (...(...) >= ...).astype(int) # TODO: Implement the necessary logic to make predictions

# Make predictions on the test set
y_pred = predict(X_test, theta)

Let's now check our model's accuracy

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Hmm. Our model did worse, why is that?

Well, remember how when we used the SKLearn model before, we specified the "ovr" parameter? That is because we were trying to predict against more than 2 classes. Because of this, we need more than 1 logistic regression model in order to make accurate results as logistic regression can only produce outputs against a binary set of classes. Our model is attempting to do the same predictions with only **one** model.

Let's do some preprocessing and retrain our model on the new data.

In [ ]:
X = iris.data[:, :2] # Using only the first two features -- will make visualizing easier later.
X = X[y < 2]  # Take only class 0 and 1
y = y[y < 2]  # Labels remain 0 and 1

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Add bias term (Intercept) to X
X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_test = np.c_[np.ones((X_test.shape[0], 1)), X_test]

Now, we are just looking at two of our classes in the dataset rather than all 3. We should get a much better result.

In [ ]:
theta, loss_history = gradient_descent(X_train, y_train, lr=0.1, epochs=1000)

# Plot loss curve
plt.plot(loss_history)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.show()

Now this time we see a better loss graph. Before, our model's loss went into the **negatives**. This is also because out function is operating with more classes than it could predict on (another indicator of the model being unable to fit). This is apart of why understanding your data is important before attempting to use it in a model.

Now that we have a corrected model, let's look at our accuracy!

In [56]:
# Make predictions on the test set
y_pred = predict(X_test, theta)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Wow! Now we are getting an accuracy of 100%! Let's take a look at the data and understand why with this model, that result makes sense.

In [ ]:
def plot_decision_boundary(X, y, theta):
    """Visualizes the decision boundary for logistic regression."""
    x_min, x_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    y_min, y_max = X[:, 2].min() - 0.5, X[:, 2].max() + 0.5

    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
    grid = np.c_[np.ones((xx.ravel().shape[0], 1)), xx.ravel(), yy.ravel()]
    probs = sigmoid(grid @ theta).reshape(xx.shape)

    plt.contourf(xx, yy, probs, levels=[0, 0.5, 1], cmap="coolwarm", alpha=0.2)
    plt.scatter(X[:, 1], X[:, 2], c=y, edgecolors="k", cmap="coolwarm")
    plt.xlabel("Feature 1 (Standardized)")
    plt.ylabel("Feature 2 (Standardized)")
    plt.title("Logistic Regression Decision Boundary")
    plt.show()

# Plot decision boundary with test data points
X_combined = np.vstack((X_train, X_test))
y_combined = np.concatenate((y_train, y_test))
plot_decision_boundary(X_combined, y_combined, theta)

So, looking at our scatter plot, we can see there is a distinct separation between our features. Because of this completely linear separation, we can form a decision boundary that entirely separates these two classes, making logistic regression a great choice for this problem!